In [1]:
import os
import sys

import warnings
warnings.filterwarnings("ignore") 

os.environ['UTILS_PATH'] = r'C:\Users\Mi\Desktop\code\git\python-utils-ak'
os.environ['PROJECT_PATH'] = r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat' 
os.environ['SQLITE_PATH'] = r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\data.sqlite' 
sys.path.append(os.environ['PROJECT_PATH'])
from app.interactive_imports import *

In [2]:
# # boiling_plan_df = read_boiling_plan(r"C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\research\akadaner\2021-01-06 (5).xlsx")
# boiling_plan_df = read_boiling_plan(r"C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\app\schedule_maker\data\sample_boiling_plan.xlsx")
# boilings = make_boilings_by_groups(boiling_plan_df)
# schedule = make_schedule(boilings, start_times={'water': '08:00', 'salt': '07:00'})
# frontend = make_frontend(schedule)
# # draw_excel_frontend(frontend, open_file=True)

In [3]:
df = read_boiling_plan(r"C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\app\schedule_maker\data\sample_boiling_plan.xlsx")
mark_consecutive_groups(df, 'boiling', 'boiling_group')
boiling_group_df = df[df['boiling_group'] == 2]
transformer = BoilingGroupToSchemaTransformer()
boilings_meltings, packings, melting_speed = transformer(boiling_group_df)
boilings_meltings, packings, melting_speed

NameError: name 'BoilingGroupToSchemaTransformer' is not defined

In [49]:
t = SchemaToBoilingsDataFramesTransformer()
t(boilings_meltings, packings, melting_speed)

Resuming 0.11222222222222222
Resuming 1.614285714285714
Resuming 2.7122023809523808


[{'meltings':           item   beg   end
  0  cheese_mass   0.0   5.0
  1  cheese_mass  10.0  70.0,
  'coolings':           item   beg   end
  0  cheese_mass   0.0   5.0
  1  cheese_mass  10.0  70.0,
  'packings':                      item   beg   end
  0   <BoilingFormFactor 3>   0.0   5.0
  1  <BoilingFormFactor 13>  10.0  70.0},
 {'meltings':           item  beg   end
  0  cheese_mass  0.0  65.0,
  'coolings':           item  beg   end
  0  cheese_mass  0.0  65.0,
  'packings':                     item  beg   end
  0  <BoilingFormFactor 4>  0.0  65.0},
 {'meltings':           item  beg   end
  0  cheese_mass  0.0  65.0,
  'coolings':           item  beg   end
  0  cheese_mass  0.0  65.0,
  'packings':                     item  beg   end
  0  <BoilingFormFactor 4>  0.0  65.0}]

In [ ]:
with make('melting'):
    boiling_model = boiling_group.iloc[0]['boiling']

    serving = make('serving', size=(boiling_model.meltings.serving_time // 5, 0), push_func=add_push).block
    meltings = make('meltings', x=(serving.size[0], 0), push_func=add_push).block
    coolings = make('coolings', x=(serving.size[0], 0), push_func=add_push).block
    
    for 
    
    for i, (group, grp) in enumerate(boiling_plan.groupby('bff_group')):
        if i >= 1 and boiling_model.boiling_type == 'water':
            # non-first group - reconfigure time
            push(meltings, maker.create_block('melting_configuration', size=(1, 0)))

        # todo: del
        if boiling_model.boiling_type == 'salt':
            boiling_model.meltings.speed = 850 / 50 * 60

        melting_process = maker.create_block('melting_process',
                                             size=(int(custom_round(grp['kg'].sum() / boiling_model.meltings.speed * 60, 5, 'ceil')) // 5, 0),
                                             bff=grp.iloc[0]['bff'])

        push(meltings, melting_process)

        # add cooling
        cooling_process = maker.create_block('cooling_process', x=melting_process.props['x_rel'])
        _, cooling_make = init_block_maker(cooling_process)

        with cooling_make('start'):
            if boiling_model.boiling_type == 'water':
                cooling_make('cooling', size=(boiling_model.meltings.first_cooling_time // 5, 0))
                cooling_make('cooling', size=(boiling_model.meltings.second_cooling_time // 5, 0))
            elif boiling_model.boiling_type == 'salt':
                cooling_make('salting', size=(boiling_model.meltings.salting_time // 5, 0))

        with cooling_make('finish'):
            if boiling_model.boiling_type == 'water':
                cooling_make('cooling', size=(melting_process.size[0], 0))
            elif boiling_model.boiling_type == 'salt':
                cooling_make('salting', size=(melting_process.size[0], 0))

        push(coolings, cooling_process, push_func=add_push)

with make('packing',
          x=(listify(maker.root['melting']['coolings']['cooling_process'])[0]['start'].y[0], 0),
          packing_team_id=1, push_func=add_push):
    for i, (_, row) in enumerate(boiling_plan.iterrows()):
        sku, kg = row['sku'], row['kg']
        packing_speed = min(sku.packing_speed, boiling_model.meltings.speed)
        make('packing_process',
             size=[int(custom_round(kg / packing_speed * 60, 5, rounding='ceil')) // 5, 0],
             sku=sku)

        if i != len(boiling_plan) - 1:
            # add configuration
            conf_time_size = get_configuration_time(boiling_model, sku, boiling_plan.iloc[i + 1]['sku'])

            if conf_time_size:
                make('packing_configuration', size=[conf_time_size // 5, 0])

return maker.root


